<a href="https://colab.research.google.com/github/weezymatt/tree-of-thoughts_demo/blob/main/src/notebooks/Tree_of_Thoughts_baselines.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tree of Thoughts

Implementation: https://github.com/princeton-nlp/tree-of-thought-llm

In [1]:
import os
import pdb
from google.colab import userdata
from pprint import pprint

os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

In [2]:
%%capture
!pip install asteval
!pip install langchain-openai

In [3]:
from langchain_openai import ChatOpenAI
import pandas as pd
from asteval import Interpreter

### Game24 Baselines — Standard Prompt

In [4]:
model_name = 'gpt-4'
llm = ChatOpenAI(temperature=0.7, model_name=model_name)

In [248]:
class Game24Task:
  """
  Game24 Class for experimenting with the various baselines in the paper.

  Baselines     : standard prompting, chain-of-thought, and self-consistency (value-based)
  """
  def __init__(self, path="https://hub.oxen.ai/api/repos/datasets/Game-of-24/file/main/24.csv"):
    self.data = pd.read_csv(path)
    self.interpreter = Interpreter()

  def __len__(self):
    return len(self.data)

  def __getitem__(self, idx):
    return self.data.iloc[idx]

  def sample(self, n=None):
    return self.data.sample(n=n)

  def standard_prompt(self, x):
    return standard_prompt.format(input=x)

  def cot_prompt(self, x):
    return cot_prompt.format(input=x)

  def parse_prompt(self, candidate_answer, prompt_type="standard_prompt"):
    errors = 0
    answer = candidate_answer.content.lower()

    if prompt_type == "standard_prompt":
      answer = answer.split("answer: ")[-1].split(" = ")[0]

    if prompt_type == "chain_of_thought":
      try:
        answer = answer.split("answer: ")[1].split(" = ")[0]
      except:
        errors += 1
        print("Error.")

    return answer

  def evaluate_expression(self, expression, puzzle):
    try:
      response = self.interpreter.eval(expression)
    except:
      response = None

    return {
        "puzzle": puzzle,
        "expression": expression,
        "answer": response,
        "valid": response == 24,
    }

In [249]:
# 5-shot
standard_prompt = '''Use numbers and basic arithmetic operations (+ - * /) to obtain 24.
Input: 4 4 6 8
Answer: (4 + 8) * (6 - 4) = 24
Input: 2 9 10 12
Answer: 2 * 12 * (10 - 9) = 24
Input: 4 9 10 13
Answer: (13 - 9) * (10 - 4) = 24
Input: 1 4 8 8
Answer: (8 / 4 + 1) * 8 = 24
Input: 5 5 5 9
Answer: 5 + 5 + 5 + 9 = 24
Input: {input}
'''

In [250]:
# simple usage
game24 = Game24Task()
x = game24.__getitem__(10)['Puzzles']
standard_prompt = game24.standard_prompt(x)

In [251]:
output = llm.invoke(standard_prompt)
expression = game24.parse_prompt(output)

In [253]:
pprint(game24.evaluate_expression(expression, x), width=50, sort_dicts=False)

{'puzzle': '1 1 2 8',
 'expression': '8 * (2 - (1 / 1))',
 'answer': 8.0,
 'valid': False}


### Game24 Baselines - Chain-of-Thought Prompt

In [254]:
# 5-shot
cot_prompt = '''Use numbers and basic arithmetic operations (+ - * /) to obtain 24. Each step, you are only allowed to choose two of the remaining numbers to obtain a new number.
Input: 4 4 6 8
Steps:
4 + 8 = 12 (left: 4 6 12)
6 - 4 = 2 (left: 2 12)
2 * 12 = 24 (left: 24)
Answer: (6 - 4) * (4 + 8) = 24
Input: 2 9 10 12
Steps:
12 * 2 = 24 (left: 9 10 24)
10 - 9 = 1 (left: 1 24)
24 * 1 = 24 (left: 24)
Answer: (12 * 2) * (10 - 9) = 24
Input: 4 9 10 13
Steps:
13 - 10 = 3 (left: 3 4 9)
9 - 3 = 6 (left: 4 6)
4 * 6 = 24 (left: 24)
Answer: 4 * (9 - (13 - 10)) = 24
Input: 1 4 8 8
Steps:
8 / 4 = 2 (left: 1 2 8)
1 + 2 = 3 (left: 3 8)
3 * 8 = 24 (left: 24)
Answer: (1 + 8 / 4) * 8 = 24
Input: 5 5 5 9
Steps:
5 + 5 = 10 (left: 5 9 10)
10 + 5 = 15 (left: 9 15)
15 + 9 = 24 (left: 24)
Answer: ((5 + 5) + 5) + 9 = 24
Input: {input}
'''

In [257]:
# simple usage
game24 = Game24Task()
x = game24.__getitem__(10)['Puzzles']
cot_prompt = game24.cot_prompt(x)

In [258]:
output = llm.invoke(cot_prompt)

In [260]:
expression = game24.parse_prompt(output, prompt_type="chain_of_thought")
pprint(game24.evaluate_expression(expression, x), width=50, sort_dicts=False)

{'puzzle': '1 1 2 8',
 'expression': '8 * 2 + 1 + 1',
 'answer': 18,
 'valid': False}


## Game24 Experiments - (Standard Prompt, CoT)

In [ ]:
def run_experiment(experiment_type="io", n_samples=100):
  prompts = []
  outputs = []
  expressions = []
  evaluations = []
  prediction = []

  samples = game24.sample(n=n_samples)['Puzzles']

  for sample in samples:
    if experiment_type == "io":
      fitted_prompt = game24.standard_prompt(sample)
    if experiment_type == "cot":
      fitted_prompt = game24.cot_prompt(sample)
    prompts.append(fitted_prompt)

  for prompt in prompts:
    output = llm.invoke(prompt)
    outputs.append(output)

  try:
    for output, sample in zip(outputs, samples):
      if experiment_type == "io":
        parsed = game24.parse_prompt(prompt_type="standard_prompt")
      if experiment_type == "cot":
        parsed = game24.parse_prompt(prompt_type="cot")

      evals = game24.evaluate_expression(parsed, sample)
      expressions.append(parsed)
      evaluations.append(evals)

  except:
    print("Expression unable to evaluate.")

  for sample in evaluations:
    prediction.append(sample['valid'])

  return {
      "samples": samples,
      "prompts": prompts,
      "outputs": outputs,
      "expressions": expressions,
      "evaluations": evaluations,
      "predictions": predictions,
  }

In [ ]:
def create_dataframe(samples_series, expressions, score):
  samples_df = samples_series.to_frame()
  results = samples_df.assign(expression=expressions, score=score)
  return results

final_results = create_dataframe(samples, expressions, predictions)
# final_results.to_csv('string.csv', index=False)

## Results on experiment

| Method     | Success   | Runs |
| --------   | --------  | ---- |
| IO prompt  | 10%       | 1    |
| CoT prompt | 26%       | 1    |
| ToT (b=?)  | ??%       | 1    |

### Game24 Baselines - Chain-of-Thought + Self-Consistency (not implemented)
"We also consider a CoT self-consistency baseline, which takes the majority output from 100 CoT samples, and an iterative-refine approach on
top of an IO sample for at most 10 iterations. At each iteration, the LM is conditioned on all previous history to “reflect on your mistakes and generate a refined answer” if the output is incorrect. Note that it uses groundtruth feedback signals about equation correctness."